# setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

# load file

In [3]:
def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    
    return result.id

# Replace with your own file path or URL
file_id = create_file(client, "https://cdn.openai.com/API/docs/deep_research_blog.pdf")

file-CCMoyvub6FNPiKTSD7s1fq


In [4]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

vs_6829770cd6108191b450ce28bc6b7a20


In [6]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)

VectorStoreFile(id='file-CCMoyvub6FNPiKTSD7s1fq', created_at=1747547994, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_6829770cd6108191b450ce28bc6b7a20', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [7]:
result = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-CCMoyvub6FNPiKTSD7s1fq', created_at=1747547917, last_error=None, object='vector_store.file', status='completed', usage_bytes=66539, vector_store_id='vs_6829770cd6108191b450ce28bc6b7a20', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-CCMoyvub6FNPiKTSD7s1fq', last_id='file-CCMoyvub6FNPiKTSD7s1fq')


In [9]:
import time
while result.data[0].status != "completed":
    print("Waiting for completion...")
    time.sleep(2)

# file search

In [10]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input="What is deep research by OpenAI?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id],
        # "max_num_results": 2
    }]
)

In [11]:
response.model_dump()

{'id': 'resp_6829778e05788191bdd53a08b54a21f003714efe75539749',
 'created_at': 1747548046.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'response',
 'output': [{'id': 'fs_6829778ea5848191a1daa6b551d263b003714efe75539749',
   'queries': ['deep research by OpenAI'],
   'status': 'completed',
   'type': 'file_search_call',
   'results': None},
  {'id': 'msg_6829779143a48191adfc0ecccac4b82103714efe75539749',
   'content': [{'annotations': [{'file_id': 'file-CCMoyvub6FNPiKTSD7s1fq',
       'index': 644,
       'type': 'file_citation',
       'filename': 'deep_research_blog.pdf'},
      {'file_id': 'file-CCMoyvub6FNPiKTSD7s1fq',
       'index': 922,
       'type': 'file_citation',
       'filename': 'deep_research_blog.pdf'},
      {'file_id': 'file-CCMoyvub6FNPiKTSD7s1fq',
       'index': 922,
       'type': 'file_citation',
       'filename': 'deep_research_blog.pdf'},
      {'file_id': 'file-CCMoyvub6